# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. Using DeepLabCut for 3D markerless pose estimation during behavior across species.

pre-print: https://www.biorxiv.org/content/10.1101/476531v1

XROMMTools for DeepLabCut written by J.D. Laurence-Chasen

This notebook walks you through the XMALab/DeepLabCut workflow, from project creation to analysis.


At this stage, you should have tracked data in XMALab and exported the 2D points for trials you wish to be included in the initial training dataset. Each trial should have its own subfolder, containing only the images (image stack or video files) and the 2D points CSV. 

In [ ]:
# Import packages. Run this every time!
import deeplabcut
from deeplabcut.utils import xrommtools

## Create a new project

The first step is to create a DeepLabCut project (network) for each camera plane. Whether or not you create new projects for each day of data collection depends in large part on the variation in the dataset and number of markers. You should always create new projects for different individuals.

N.B. If you are creating a new project for a different day of data collection on the same animal (or an animal with a similar marker scheme), it may be easier to manually copy an existing project and delete the folder contents to minimize config file editing.

In [ ]:
# Run this section only once

task='PigFeeding' # Enter the name of your experiment Task
experimenter='JD' # Enter the name of the experimenter
video=['C:/Users/jdlc7/PigDLC/pig20190227_cam1.avi'] # IMPORTANT: This video is a 'dummy' video. Just point the function to something. A future version will eliminate this step.
working_directory = 'C:/Users/jdlc7/PigDLC_cam1' # where the project will be created; use forward slashes on Windows

path_config_file_cam1=deeplabcut.create_new_project(task,experimenter,video,working_directory,copy_videos=False)

# repeat for second camera
video=['C:/Users/jdlc7/PigDLC/pig20190227_cam2.avi'] 
working_directory = 'C:/Users/jdlc7/PigDLC_cam2'
path_config_file_cam2=deeplabcut.create_new_project(task,experimenter,video,working_directory,copy_videos=False)

Now edit the newly created configuration file to match your project. See DLC methods paper for the significance of the different terms.

ESSENTIAL: Make sure that video_sets: contains the dummy video (path doesn't matter) whose filename is identical to the name of your training dataset. E.g. 'C:/PigDLC/pig20190227.avi'. This dataset name (pig20190227) will stay consistent throughout the iterative retraining of the two networks. 

In [ ]:
# Set working directory paths for cam1 and cam2, if you haven't yet

path_config_file_cam1 = 'C:/Users/jdlc7/PigDLC_cam1/PigFeeding-JD-2019-02-27/config.yaml'
path_config_file_cam2 = 'C:/Users/jdlc7/PigDLC_cam2/PigFeeding-JD-2019-02-27/config.yaml'

## Convert XMALab 2D points to DLC format

Now we generate a training dataset from the 2D points file/s exported from XMALab and frames from the videos. The function xma2dlc will populate each projects labeled-data folder with the extracted data.

In [14]:
experimenter = 'JD'
data_path = 'C:/Users/jdlc7/Pig20190227/DLC' # where are the trial folders (with 2D points/trial images)
dataset_name = 'pig20190227' # IMPORTANT, omit the _cam1/_cam2
nframes = 400 # how many frames do you want the training dataset to comprise?

xrommtools.xma2dlc(path_config_file_cam1,path_config_file_cam2,data_path,dataset_name,experimenter,nframes)

## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typically, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [ ]:
deeplabcut.create_training_dataset(path_config_file_cam1)
deeplabcut.create_training_dataset(path_config_file_cam2)

## Start training:

This function trains the network for a specific shuffle of the training dataset. This will take a long time. You can stop training manually by hitting stop. It will look like an error has occured...it has not. 

In [ ]:
deeplabcut.train_network(path_config_file_cam1,maxiters = 500000)

tf.reset_network()

deeplabcut.train_network(path_config_file_cam2,maxiters = 500000)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**. The values here can give you an indication of performance/generalizability, but are less informative than reprojection error and other XMALab metrics.

In [ ]:
deeplabcut.evaluate_network(path_config_file_cam1, plotting=True)
deeplabcut.evaluate_network(path_config_file_cam2, plotting=True)

## Analyzing videos
This function uses the existing DLC function analyze_videos to predict points for new trials and convert the output to XMALab format. The results are stored in hd5 file and CSV file in each trial's specific iteration subfolder.

The base function analyze_videos has many possible arguments. To alter this, you will need to edit the xrommtools script manually. Its default form should be sufficient in most cases.

In [6]:
new_data_path = '' # where are the trials you wish to analyze? 
# IMPORTANT: data_path must only contain trial folders. Any other files/folders will cause an error

iteration = 0 # what iteration of the network do you want to analyze the videos?

xrommtools.analyze_xromm_videos(path_config_file_cam1,path_config_file_cam2,new_data_path,iteration)

## Create a new iteration of training dataset [optional step]
Following the correction of frames in XMALab and the re-exporting of 2D points, you can augment the training dataset and retrain the networks.

In [ ]:
frames = 'C:/Users/jdlc7/PigDLC/frameindex.csv' # column 1 should be trialnames, and columns 2+ should be frame numbers

# data_path = where are trial folders with new videos to analyze?

xrommtools.add_frames(path_config_file_cam1, path_config_file_cam2, data_path, iteration, frames)


In [ ]:
# IMPORTANT!!!: Before creating new training dataset, update the iteration value in the two configuration files (i.e. 0 --> 1).

deeplabcut.create_training_dataset(path_config_file)
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [ ]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)